In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 101 kB 4.5 MB/s 
     |████████████████████████████████| 596 kB 36.9 MB/s 
     |████████████████████████████████| 6.6 MB 63.3 MB/s 


In [2]:
!pip install -q datasets==2.0.0

     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 141 kB 70.0 MB/s 
     |████████████████████████████████| 212 kB 77.9 MB/s 
     |████████████████████████████████| 115 kB 85.0 MB/s 
     |████████████████████████████████| 127 kB 86.5 MB/s 


In [3]:
!pip install -q seqeval


     |████████████████████████████████| 43 kB 1.4 MB/s 


In [4]:
!pip install -q pyyaml==5.1
# workaround: install old version of pytorch since detectron2 hasn't released packages for pytorch 1.9 (issue: https://github.com/facebookresearch/detectron2/issues/3158)
!pip install -q torch==1.11.0+cu113 torchvision==0.9.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# install detectron2 that matches pytorch 1.8
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install -q detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

#!git clone https://github.com/facebookresearch/detectron2.git
#!python -m pip install -e detectron2

     |████████████████████████████████| 274 kB 4.9 MB/s 
     |████████████████▎               | 834.1 MB 1.2 MB/s eta 0:11:34tcmalloc: large alloc 1147494400 bytes == 0x3a7b6000 @  0x7f7d1850b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████▋           | 1055.7 MB 1.2 MB/s eta 0:07:49tcmalloc: large alloc 1434370048 bytes == 0x7ee0c000 @  0x7f7d1850b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |██████████████████████████▏     | 1336.2 MB 1.2 MB/s eta 0:04:09tcmalloc: large alloc 1792966656 bytes == 0x3c3e000 

In [27]:
#imports
import os
import glob
import json
import random
from pathlib import Path
from difflib import SequenceMatcher

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8" 
import torch 
import numpy as np
import random
seed = 7
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.use_deterministic_algorithms(True)

import cv2
import pandas as pd
from PIL import Image
from tqdm import tqdm
from IPython.display import display
import matplotlib
from PIL import Image
from matplotlib import pyplot, patches
from transformers import LayoutLMv2Tokenizer, LayoutLMModel
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
from torch.utils.data import DataLoader
from transformers import LayoutLMForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric


In [28]:


datasets = load_dataset("darentang/sroie",cache_dir=None)
#datasets = load_dataset("nielsr/funsd",cache_dir=None)
#datasets = load_dataset("nielsr/funsd-layoutlmv3")

##creating a dataloader

from torch.utils.data import DataLoader

# train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=2)


Reusing dataset sroie (/root/.cache/huggingface/datasets/darentang___sroie/sroie/1.0.0/26ed9374c9a15a1d2f44fd8886f679076e1a1fd7da2d53726d6e58a99436c506)


  0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
#labels
labels = datasets['train'].features['ner_tags'].feature.names
print(labels)
#labels
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}
label2id


['O', 'B-COMPANY', 'I-COMPANY', 'B-DATE', 'I-DATE', 'B-ADDRESS', 'I-ADDRESS', 'B-TOTAL', 'I-TOTAL']


{'B-ADDRESS': 5,
 'B-COMPANY': 1,
 'B-DATE': 3,
 'B-TOTAL': 7,
 'I-ADDRESS': 6,
 'I-COMPANY': 2,
 'I-DATE': 4,
 'I-TOTAL': 8,
 'O': 0}

#Text: The original word text.

Lemma: The base form of the word.

POS: The simple UPOS part-of-speech tag.

Tag: The detailed part-of-speech tag.

Dep: Syntactic dependency, i.e. the relation between tokens.

Shape: The word shape – capitalization, punctuation, digits.

is alpha: Is the token an alpha character?

is stop: Is the token part of a stop list, i.e. the most common words of the language?

In [30]:
datasets["train"]

Dataset({
    features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
    num_rows: 626
})

In [31]:


tokenizer =  LayoutLMv2Tokenizer.from_pretrained("microsoft/layoutlmv2-base-uncased")


def encode_example(example, max_seq_length=512, pad_token_box=[0, 0, 0, 0]):

  
  encoded_inputs = tokenizer(text = example["words"],
            boxes=example["bboxes"],
            word_labels=example["ner_tags"],
            add_special_tokens=True,
            padding=True,
            truncation=True,
            max_length=512,
        )
  #encoded_inputs["labels"] =  [torch.reshape(x, (x.shape[0], -1)) for x in encoded_inputs["labels"]] 
  
  return encoded_inputs

# we need to define the features ourselves as the bbox of LayoutLM are an extra feature
features = Features({
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'labels':  Sequence(ClassLabel(names=labels))
})

train_dataset = datasets["train"].map( encode_example,remove_columns=datasets["train"].column_names,features=features, batched=True)
test_dataset = datasets["test"].map(encode_example,remove_columns=datasets["test"].column_names,features=features,batched=True)



# train_dataset = datasets["train"].map(encode_example, batched=True, remove_columns=datasets["train"].column_names, features=features)
# test_dataset = datasets["test"].map(encode_example, batched=True, remove_columns=datasets["test"].column_names,features=features)

Loading cached processed dataset at /root/.cache/huggingface/datasets/darentang___sroie/sroie/1.0.0/26ed9374c9a15a1d2f44fd8886f679076e1a1fd7da2d53726d6e58a99436c506/cache-f2a74de452e6b438.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/darentang___sroie/sroie/1.0.0/26ed9374c9a15a1d2f44fd8886f679076e1a1fd7da2d53726d6e58a99436c506/cache-6513270e269e0d37.arrow


In [32]:
train_dataset.set_format(type="torch")
test_dataset.set_format(type="torch")

In [33]:

##creating a dataloader



train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset,  batch_size=8)

In [35]:


model =  LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased",num_labels=len(label2id))

# Set id2label and label2id 
model.config.id2label = id2label
model.config.label2id = label2id

# Metrics
metric = load_metric("seqeval")
return_entity_level_metrics = True



Some weights of the model checkpoint at microsoft/layoutlm-base-uncased were not used when initializing LayoutLMForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing LayoutLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [37]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

class FunsdTrainer(Trainer):
    def get_train_dataloader(self):
      return train_dataloader

    def get_test_dataloader(self, test_dataset):
      return test_dataloader


In [38]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(model))

112634889


In [39]:
args = TrainingArguments(
    output_dir="layoutlm-finetuned-funsd-v2", # name of directory to store the checkpoints
    max_steps=1000, # we train for a maximum of 1,000 batches
    warmup_ratio=0.1, # we warmup a bit
    fp16=True # we use mixed precision (less memory consumption)
    , # after training, we'd like to push our model to the hub
     # this is the name we'll use for our model on the hub
)

# Initialize our Trainer
trainer = FunsdTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
)



max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend


In [40]:
trainer.train()


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 626
  Num Epochs = 13
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000


Step,Training Loss
500,0.159100
1000,0.003500


Saving model checkpoint to layoutlm-finetuned-funsd-v2/checkpoint-500
Configuration saved in layoutlm-finetuned-funsd-v2/checkpoint-500/config.json
Model weights saved in layoutlm-finetuned-funsd-v2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to layoutlm-finetuned-funsd-v2/checkpoint-1000
Configuration saved in layoutlm-finetuned-funsd-v2/checkpoint-1000/config.json
Model weights saved in layoutlm-finetuned-funsd-v2/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1000, training_loss=0.08130487060546875, metrics={'train_runtime': 472.1638, 'train_samples_per_second': 16.943, 'train_steps_per_second': 2.118, 'total_flos': 2086075548868608.0, 'train_loss': 0.08130487060546875, 'epoch': 12.66})

In [41]:
predictions, labels, metrics = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 347
  Batch size = 8


In [ ]:
metrics

In [ ]:
## layoutlm 11 M base, 2 epoch